**# INSTALLATION**

In [ ]:
!pip install langchain langchain-openai langchain-community langchain-core networkx matplotlib plotly pandas numpy python-dotenv tiktoken redis

# IMPORTS

import os
import json
import sqlite3
import hashlib
import networkx as nx
from datetime import datetime, timedelta
from typing import Dict, List, Any, Optional, Union, Tuple
from dataclasses import dataclass, field
from enum import Enum
import pandas as pd
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# LangChain imports
from langchain.agents import AgentExecutor, create_react_agent
from langchain.tools import tool
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema import AgentAction, AgentFinish
from langchain.callbacks.base import BaseCallbackHandler

# Load environment variables
from dotenv import load_dotenv
load_dotenv()

# Set your OpenAI API key here
os.environ["OPENAI_API_KEY"] = "your-api-key-here"


**# SECTION 1: Query Decomposition and Planning (13.3)**

In [ ]:
print("="*60)
print("SECTION 1: QUERY DECOMPOSITION AND PLANNING")
print("="*60)

class QueryType(Enum):
    """Classification of query types for appropriate planning strategies"""
    FACTUAL = "factual"
    ANALYTICAL = "analytical"
    COMPARATIVE = "comparative"
    RESEARCH = "research"
    SYNTHESIS = "synthesis"

@dataclass
class PlanningStep:
    """Individual step in a multi-step plan"""
    id: str
    description: str
    step_type: str
    dependencies: List[str] = field(default_factory=list)
    tools_required: List[str] = field(default_factory=list)
    expected_output: str = ""
    priority: int = 1
    estimated_time: int = 30  # seconds
    status: str = "pending"  # pending, in_progress, completed, failed
    result: Any = None
    confidence: float = 0.0

@dataclass
class ExecutionPlan:
    """Complete execution plan for complex queries"""
    query: str
    query_type: QueryType
    steps: List[PlanningStep]
    execution_order: List[str]
    parallel_groups: List[List[str]] = field(default_factory=list)
    total_estimated_time: int = 0
    created_at: datetime = field(default_factory=datetime.now)

class QueryDecomposer:
    """Advanced query decomposition system"""

    def __init__(self, llm):
        self.llm = llm
        self.decomposition_prompt = PromptTemplate.from_template("""
You are an expert query analyst. Decompose the following complex query into specific, actionable steps.

Query: {query}
Query Type: {query_type}

For each step, provide:
1. Step description (what to do)
2. Step type (search, analyze, calculate, synthesize, verify)
3. Required tools (web_search, database, calculator, etc.)
4. Dependencies (which previous steps must complete first)
5. Expected output type
6. Priority (1=critical, 2=important, 3=optional)

Consider:
- Information gathering needs
- Analysis requirements
- Verification steps
- Synthesis and presentation needs
- Potential parallel execution opportunities

Respond in JSON format:
{{
    "steps": [
        {{
            "id": "step_1",
            "description": "Search for recent information about...",
            "step_type": "search",
            "dependencies": [],
            "tools_required": ["web_search"],
            "expected_output": "List of recent developments",
            "priority": 1
        }}
    ],
    "parallel_groups": [["step_1", "step_2"], ["step_3"]],
    "execution_strategy": "sequential_with_parallel_opportunities"
}}
        """)

    def classify_query(self, query: str) -> QueryType:
        """Classify query to determine appropriate planning strategy"""
        classification_prompt = f"""
        Classify this query into one category:
        - factual: Simple fact-finding
        - analytical: Requires analysis of data/trends
        - comparative: Comparing options/alternatives
        - research: Comprehensive investigation
        - synthesis: Combining multiple sources/perspectives

        Query: {query}

        Return only the category name.
        """

        response = self.llm.invoke(classification_prompt)
        try:
            return QueryType(response.content.strip().lower())
        except ValueError:
            return QueryType.RESEARCH  # Default fallback

    def decompose_query(self, query: str) -> ExecutionPlan:
        """Decompose complex query into structured execution plan"""
        query_type = self.classify_query(query)

        prompt = self.decomposition_prompt.format(
            query=query,
            query_type=query_type.value
        )

        response = self.llm.invoke(prompt)

        try:
            plan_data = json.loads(response.content)

            # Convert to PlanningStep objects
            steps = []
            for step_data in plan_data["steps"]:
                step = PlanningStep(
                    id=step_data["id"],
                    description=step_data["description"],
                    step_type=step_data["step_type"],
                    dependencies=step_data.get("dependencies", []),
                    tools_required=step_data.get("tools_required", []),
                    expected_output=step_data.get("expected_output", ""),
                    priority=step_data.get("priority", 1)
                )
                steps.append(step)

            # Determine execution order
            execution_order = self._determine_execution_order(steps)

            # Extract parallel groups
            parallel_groups = plan_data.get("parallel_groups", [])

            return ExecutionPlan(
                query=query,
                query_type=query_type,
                steps=steps,
                execution_order=execution_order,
                parallel_groups=parallel_groups,
                total_estimated_time=sum(step.estimated_time for step in steps)
            )

        except (json.JSONDecodeError, KeyError) as e:
            print(f"Error parsing decomposition response: {e}")
            # Fallback to simple plan
            return self._create_fallback_plan(query, query_type)

    def _determine_execution_order(self, steps: List[PlanningStep]) -> List[str]:
        """Determine optimal execution order based on dependencies"""
        # Create dependency graph
        graph = nx.DiGraph()

        for step in steps:
            graph.add_node(step.id)
            for dep in step.dependencies:
                graph.add_edge(dep, step.id)

        try:
            # Topological sort for dependency order
            return list(nx.topological_sort(graph))
        except nx.NetworkXError:
            # Fallback to original order if cycles detected
            return [step.id for step in steps]

    def _create_fallback_plan(self, query: str, query_type: QueryType) -> ExecutionPlan:
        """Create simple fallback plan when decomposition fails"""
        fallback_step = PlanningStep(
            id="fallback_step",
            description=f"Process query: {query}",
            step_type="general",
            tools_required=["web_search", "calculator"],
            expected_output="Response to user query"
        )

        return ExecutionPlan(
            query=query,
            query_type=query_type,
            steps=[fallback_step],
            execution_order=["fallback_step"]
        )

class PlanExecutor:
    """Execute multi-step plans with monitoring and adaptation"""

    def __init__(self, llm, tools: List):
        self.llm = llm
        self.tools = {tool.name: tool for tool in tools}
        self.execution_history = []
        self.active_plans = {}

    def execute_plan(self, plan: ExecutionPlan, adaptive: bool = True) -> Dict[str, Any]:
        """Execute a complete plan with optional adaptation"""
        print(f"\n🎯 Executing plan for: {plan.query}")
        print(f"Plan type: {plan.query_type.value}")
        print(f"Total steps: {len(plan.steps)}")
        print(f"Estimated time: {plan.total_estimated_time}s")

        execution_results = {
            "plan_id": id(plan),
            "query": plan.query,
            "steps_completed": 0,
            "steps_failed": 0,
            "total_steps": len(plan.steps),
            "start_time": datetime.now(),
            "step_results": {},
            "final_synthesis": None,
            "success": False
        }

        self.active_plans[id(plan)] = plan

        try:
            # Execute steps in order
            for step_id in plan.execution_order:
                step = next(s for s in plan.steps if s.id == step_id)

                # Check dependencies
                if not self._dependencies_satisfied(step, execution_results["step_results"]):
                    print(f"⏸️ Skipping {step_id} - dependencies not satisfied")
                    continue

                print(f"\n▶️ Executing step: {step.description}")
                step.status = "in_progress"

                step_result = self._execute_step(step, execution_results["step_results"])

                if step_result["success"]:
                    step.status = "completed"
                    step.result = step_result["result"]
                    step.confidence = step_result.get("confidence", 0.0)
                    execution_results["steps_completed"] += 1
                    print(f"✅ Step completed with confidence: {step.confidence:.2f}")
                else:
                    step.status = "failed"
                    execution_results["steps_failed"] += 1
                    print(f"❌ Step failed: {step_result.get('error', 'Unknown error')}")

                    if adaptive:
                        # Attempt adaptation
                        adaptation_result = self._adapt_plan(plan, step, step_result)
                        if adaptation_result:
                            print(f"🔄 Plan adapted successfully")

                execution_results["step_results"][step_id] = step_result

            # Synthesize final results
            if execution_results["steps_completed"] > 0:
                synthesis_result = self._synthesize_results(plan, execution_results["step_results"])
                execution_results["final_synthesis"] = synthesis_result
                execution_results["success"] = True

            execution_results["end_time"] = datetime.now()
            execution_results["total_time"] = (execution_results["end_time"] - execution_results["start_time"]).total_seconds()

            self.execution_history.append(execution_results)

            return execution_results

        except Exception as e:
            print(f"❌ Plan execution failed: {e}")
            execution_results["error"] = str(e)
            execution_results["end_time"] = datetime.now()
            return execution_results

        finally:
            if id(plan) in self.active_plans:
                del self.active_plans[id(plan)]

    def _dependencies_satisfied(self, step: PlanningStep, completed_results: Dict) -> bool:
        """Check if all step dependencies are satisfied"""
        return all(dep in completed_results and
                  completed_results[dep].get("success", False)
                  for dep in step.dependencies)

    def _execute_step(self, step: PlanningStep, context: Dict) -> Dict[str, Any]:
        """Execute individual planning step"""
        try:
            # Select appropriate tool
            if not step.tools_required:
                # Use general reasoning if no specific tools required
                return self._general_reasoning_step(step, context)

            # Try each required tool until success
            for tool_name in step.tools_required:
                if tool_name in self.tools:
                    tool = self.tools[tool_name]

                    # Prepare input for tool
                    tool_input = self._prepare_tool_input(step, context, tool_name)

                    try:
                        result = tool.func(tool_input)
                        confidence = self._estimate_confidence(result, step)

                        return {
                            "success": True,
                            "result": result,
                            "tool_used": tool_name,
                            "confidence": confidence,
                            "step_id": step.id
                        }
                    except Exception as tool_error:
                        print(f"Tool {tool_name} failed: {tool_error}")
                        continue

            return {
                "success": False,
                "error": f"No working tools available from: {step.tools_required}",
                "step_id": step.id
            }

        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "step_id": step.id
            }

    def _prepare_tool_input(self, step: PlanningStep, context: Dict, tool_name: str) -> str:
        """Prepare appropriate input for specific tool based on step and context"""
        # Extract relevant context from previous steps
        context_summary = ""
        if context:
            context_summary = "\n".join([
                f"Previous step result: {result.get('result', '')[:200]}..."
                for result in context.values() if result.get("success")
            ])

        # Create tool-specific input
        if tool_name == "web_search_tool":
            return step.description.replace("Search for", "").replace("Find", "").strip()
        elif tool_name == "calculator_tool":
            # Extract calculation from description
            return step.description
        else:
            # Generic input
            return f"{step.description}\n\nContext: {context_summary}"

    def _general_reasoning_step(self, step: PlanningStep, context: Dict) -> Dict[str, Any]:
        """Execute step using general LLM reasoning"""
        reasoning_prompt = f"""
        Execute this reasoning step: {step.description}

        Context from previous steps:
        {json.dumps({k: v.get('result', '') for k, v in context.items()}, indent=2)}

        Expected output: {step.expected_output}

        Provide a clear, specific response.
        """

        try:
            response = self.llm.invoke(reasoning_prompt)
            return {
                "success": True,
                "result": response.content,
                "tool_used": "llm_reasoning",
                "confidence": 0.7,  # Default confidence for reasoning
                "step_id": step.id
            }
        except Exception as e:
            return {
                "success": False,
                "error": str(e),
                "step_id": step.id
            }

    def _estimate_confidence(self, result: str, step: PlanningStep) -> float:
        """Estimate confidence in step result"""
        # Simple heuristic-based confidence estimation
        confidence = 0.5  # Base confidence

        # Length-based confidence (longer results often more informative)
        if len(str(result)) > 100:
            confidence += 0.2

        # Step type based confidence
        if step.step_type in ["search", "calculate"]:
            confidence += 0.2
        elif step.step_type in ["analyze", "synthesize"]:
            confidence += 0.1

        # Keyword-based confidence indicators
        positive_indicators = ["found", "discovered", "confirmed", "verified", "calculated"]
        negative_indicators = ["not found", "unclear", "uncertain", "failed", "error"]

        result_lower = str(result).lower()
        for indicator in positive_indicators:
            if indicator in result_lower:
                confidence += 0.1
                break

        for indicator in negative_indicators:
            if indicator in result_lower:
                confidence -= 0.2
                break

        return max(0.0, min(1.0, confidence))

    def _adapt_plan(self, plan: ExecutionPlan, failed_step: PlanningStep, error_result: Dict) -> bool:
        """Adapt plan when step fails"""
        adaptation_prompt = f"""
        A planning step failed. Suggest adaptations:

        Failed step: {failed_step.description}
        Error: {error_result.get('error', 'Unknown error')}
        Original tools: {failed_step.tools_required}
        Available tools: {list(self.tools.keys())}

        Suggest:
        1. Alternative tools to try
        2. Modified step description
        3. Whether to skip this step

        Respond with JSON: {{"action": "retry|modify|skip", "new_tools": [], "new_description": ""}}
        """

        try:
            response = self.llm.invoke(adaptation_prompt)
            adaptation = json.loads(response.content)

            if adaptation["action"] == "retry" and adaptation["new_tools"]:
                failed_step.tools_required = adaptation["new_tools"]
                return True
            elif adaptation["action"] == "modify":
                failed_step.description = adaptation.get("new_description", failed_step.description)
                failed_step.tools_required = adaptation.get("new_tools", failed_step.tools_required)
                return True

        except Exception as e:
            print(f"Adaptation failed: {e}")

        return False

    def _synthesize_results(self, plan: ExecutionPlan, step_results: Dict) -> str:
        """Synthesize results from all completed steps"""
        synthesis_prompt = f"""
        Synthesize the following step results into a comprehensive answer for the original query.

        Original Query: {plan.query}

        Step Results:
        {json.dumps({k: v.get('result', '') for k, v in step_results.items() if v.get('success')}, indent=2)}

        Provide a clear, comprehensive answer that:
        1. Directly addresses the original query
        2. Integrates information from multiple steps
        3. Highlights key findings and insights
        4. Notes any limitations or uncertainties
        """

        try:
            response = self.llm.invoke(synthesis_prompt)
            return response.content
        except Exception as e:
            return f"Synthesis failed: {e}"

**# SECTION 2: Memory Systems and Context Management (13.5)**

In [ ]:
print("\n" + "="*60)
print("SECTION 2: MEMORY SYSTEMS AND CONTEXT MANAGEMENT")
print("="*60)

class MemoryType(Enum):
    """Types of memory in agentic systems"""
    WORKING = "working"          # Short-term task memory
    EPISODIC = "episodic"        # Memory of past interactions
    SEMANTIC = "semantic"        # Factual knowledge
    PROCEDURAL = "procedural"    # Learned procedures and strategies

@dataclass
class MemoryEntry:
    """Individual memory entry"""
    id: str
    memory_type: MemoryType
    content: Any
    context: Dict[str, Any]
    timestamp: datetime
    access_count: int = 0
    last_accessed: Optional[datetime] = None
    importance: float = 0.5
    tags: List[str] = field(default_factory=list)

    def access(self):
        """Record memory access"""
        self.access_count += 1
        self.last_accessed = datetime.now()

class MemoryManager:
    """Advanced memory management system for agentic RAG"""

    def __init__(self, db_path: str = ":memory:", max_working_memory: int = 50):
        self.db_path = db_path
        self.max_working_memory = max_working_memory
        self.working_memory = {}  # Recent context
        self.importance_threshold = 0.3

        # Initialize database
        self._init_database()

        # Memory consolidation settings
        self.consolidation_interval = timedelta(hours=1)
        self.last_consolidation = datetime.now()

    def _init_database(self):
        """Initialize SQLite database for persistent memory"""
        self.conn = sqlite3.connect(self.db_path, check_same_thread=False)
        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS memory_entries (
                id TEXT PRIMARY KEY,
                memory_type TEXT NOT NULL,
                content TEXT NOT NULL,
                context TEXT,
                timestamp TEXT NOT NULL,
                access_count INTEGER DEFAULT 0,
                last_accessed TEXT,
                importance REAL DEFAULT 0.5,
                tags TEXT
            )
        """)

        self.conn.execute("""
            CREATE TABLE IF NOT EXISTS memory_relationships (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                source_id TEXT NOT NULL,
                target_id TEXT NOT NULL,
                relationship_type TEXT NOT NULL,
                strength REAL DEFAULT 1.0,
                created_at TEXT NOT NULL,
                FOREIGN KEY (source_id) REFERENCES memory_entries (id),
                FOREIGN KEY (target_id) REFERENCES memory_entries (id)
            )
        """)

        self.conn.commit()

    def store_memory(self, memory_type: MemoryType, content: Any,
                    context: Dict = None, tags: List[str] = None) -> str:
        """Store new memory entry"""
        memory_id = hashlib.md5(f"{content}{datetime.now()}".encode()).hexdigest()

        entry = MemoryEntry(
            id=memory_id,
            memory_type=memory_type,
            content=content,
            context=context or {},
            timestamp=datetime.now(),
            tags=tags or []
        )

        # Store in working memory if appropriate
        if memory_type == MemoryType.WORKING:
            self.working_memory[memory_id] = entry
            self._manage_working_memory_size()

        # Store in persistent database
        self._store_persistent_memory(entry)

        return memory_id

    def _store_persistent_memory(self, entry: MemoryEntry):
        """Store memory entry in persistent database"""
        self.conn.execute("""
            INSERT OR REPLACE INTO memory_entries
            (id, memory_type, content, context, timestamp, access_count,
             last_accessed, importance, tags)
            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
        """, (
            entry.id,
            entry.memory_type.value,
            json.dumps(entry.content) if not isinstance(entry.content, str) else entry.content,
            json.dumps(entry.context),
            entry.timestamp.isoformat(),
            entry.access_count,
            entry.last_accessed.isoformat() if entry.last_accessed else None,
            entry.importance,
            json.dumps(entry.tags)
        ))
        self.conn.commit()

    def retrieve_memory(self, query: str, memory_types: List[MemoryType] = None,
                       limit: int = 10) -> List[MemoryEntry]:
        """Retrieve relevant memories based on query"""
        if memory_types is None:
            memory_types = list(MemoryType)

        type_filter = " OR ".join([f"memory_type = '{mt.value}'" for mt in memory_types])

        # Simple keyword matching (in production, use vector similarity)
        query_words = query.lower().split()

        results = self.conn.execute(f"""
            SELECT * FROM memory_entries
            WHERE ({type_filter})
            ORDER BY importance DESC, access_count DESC, timestamp DESC
            LIMIT ?
        """, (limit * 2,)).fetchall()  # Get more to filter

        # Convert to MemoryEntry objects and filter by relevance
        relevant_memories = []
        for row in results:
            entry = self._row_to_memory_entry(row)

            # Simple relevance scoring
            content_lower = str(entry.content).lower()
            context_lower = json.dumps(entry.context).lower()
            tags_lower = " ".join(entry.tags).lower()

            relevance_score = 0
            for word in query_words:
                if word in content_lower:
                    relevance_score += 2
                if word in context_lower:
                    relevance_score += 1
                if word in tags_lower:
                    relevance_score += 1

            if relevance_score > 0 or entry.memory_type == MemoryType.WORKING:
                entry.access()  # Record access
                self._update_memory_access(entry)
                relevant_memories.append(entry)

        return sorted(relevant_memories,
                     key=lambda x: (x.importance, x.access_count),
                     reverse=True)[:limit]

    def _row_to_memory_entry(self, row) -> MemoryEntry:
        """Convert database row to MemoryEntry object"""
        return MemoryEntry(
            id=row[0],
            memory_type=MemoryType(row[1]),
            content=json.loads(row[2]) if row[2].startswith(('{', '[')) else row[2],
            context=json.loads(row[3]) if row[3] else {},
            timestamp=datetime.fromisoformat(row[4]),
            access_count=row[5] or 0,
            last_accessed=datetime.fromisoformat(row[6]) if row[6] else None,
            importance=row[7] or 0.5,
            tags=json.loads(row[8]) if row[8] else []
        )

    def _update_memory_access(self, entry: MemoryEntry):
        """Update memory access statistics in database"""
        self.conn.execute("""
            UPDATE memory_entries
            SET access_count = ?, last_accessed = ?
            WHERE id = ?
        """, (entry.access_count, entry.last_accessed.isoformat(), entry.id))
        self.conn.commit()

    def _manage_working_memory_size(self):
        """Manage working memory size by removing least important entries"""
        if len(self.working_memory) > self.max_working_memory:
            # Sort by importance and recency
            sorted_entries = sorted(
                self.working_memory.items(),
                key=lambda x: (x[1].importance, x[1].timestamp),
                reverse=True
            )

            # Keep most important entries
            keep_entries = dict(sorted_entries[:self.max_working_memory])

            # Move removed entries to long-term memory if important enough
            for entry_id, entry in self.working_memory.items():
                if entry_id not in keep_entries and entry.importance > self.importance_threshold:
                    # Convert to episodic memory
                    entry.memory_type = MemoryType.EPISODIC
                    self._store_persistent_memory(entry)

            self.working_memory = keep_entries

    def consolidate_memory(self, llm):
        """Periodic memory consolidation to extract insights and patterns"""
        if datetime.now() - self.last_consolidation < self.consolidation_interval:
            return

        print("🧠 Starting memory consolidation...")

        # Get recent memories for consolidation
        recent_cutoff = datetime.now() - timedelta(hours=24)
        recent_memories = self.conn.execute("""
            SELECT * FROM memory_entries
            WHERE timestamp > ? AND memory_type IN ('working', 'episodic')
            ORDER BY importance DESC, access_count DESC
            LIMIT 20
        """, (recent_cutoff.isoformat(),)).fetchall()

        if len(recent_memories) < 3:
            return

        # Extract patterns and insights
        memory_contents = []
        for row in recent_memories:
            entry = self._row_to_memory_entry(row)
            memory_contents.append({
                "content": entry.content,
                "context": entry.context,
                "tags": entry.tags
            })

        consolidation_prompt = f"""
        Analyze these recent memory entries and extract:
        1. Common patterns or themes
        2. Important facts that should be remembered long-term
        3. Procedural knowledge or strategies that worked well
        4. Connections between different pieces of information

        Memory entries:
        {json.dumps(memory_contents, indent=2)}

        Provide response as JSON:
        {{
            "patterns": ["pattern1", "pattern2"],
            "important_facts": ["fact1", "fact2"],
            "procedures": ["procedure1", "procedure2"],
            "connections": [["item1", "item2", "relationship"]]
        }}
        """

        try:
            response = llm.invoke(consolidation_prompt)
            insights = json.loads(response.content)

            # Store consolidated insights as semantic memory
            for pattern in insights.get("patterns", []):
                self.store_memory(
                    MemoryType.SEMANTIC,
                    f"Pattern: {pattern}",
                    {"source": "consolidation", "type": "pattern"},
                    ["pattern", "insight"]
                )

            for fact in insights.get("important_facts", []):
                self.store_memory(
                    MemoryType.SEMANTIC,
                    f"Fact: {fact}",
                    {"source": "consolidation", "type": "fact"},
                    ["fact", "knowledge"]
                )

            for procedure in insights.get("procedures", []):
                self.store_memory(
                    MemoryType.PROCEDURAL,
                    f"Procedure: {procedure}",
                    {"source": "consolidation", "type": "procedure"},
                    ["procedure", "strategy"]
                )

            # Create memory relationships
            for connection in insights.get("connections", []):
                if len(connection) >= 3:
                    self._create_memory_relationship(connection[0], connection[1], connection[2])

            self.last_consolidation = datetime.now()
            print(f"✅ Consolidated {len(recent_memories)} memories into {len(insights.get('patterns', [])) + len(insights.get('important_facts', [])) + len(insights.get('procedures', []))} insights")

        except Exception as e:
            print(f"❌ Memory consolidation failed: {e}")

    def _create_memory_relationship(self, source_content: str, target_content: str, relationship: str):
        """Create relationship between memory entries"""
        # Find memory IDs by content (simplified - in production use better matching)
        source_ids = self.conn.execute("""
            SELECT id FROM memory_entries WHERE content LIKE ?
        """, (f"%{source_content}%",)).fetchall()

        target_ids = self.conn.execute("""
            SELECT id FROM memory_entries WHERE content LIKE ?
        """, (f"%{target_content}%",)).fetchall()

        if source_ids and target_ids:
            self.conn.execute("""
                INSERT INTO memory_relationships
                (source_id, target_id, relationship_type, created_at)
                VALUES (?, ?, ?, ?)
            """, (source_ids[0][0], target_ids[0][0], relationship, datetime.now().isoformat()))
            self.conn.commit()

    def get_memory_context(self, query: str, max_entries: int = 5) -> str:
        """Get formatted memory context for agent prompts"""
        relevant_memories = self.retrieve_memory(query, limit=max_entries)

        if not relevant_memories:
            return "No relevant previous context found."

        context_parts = []
        context_parts.append("=== RELEVANT MEMORY CONTEXT ===")

        for memory in relevant_memories:
            context_parts.append(f"\n[{memory.memory_type.value.upper()}] {memory.content}")
            if memory.tags:
                context_parts.append(f"Tags: {', '.join(memory.tags)}")

        context_parts.append("\n=== END CONTEXT ===")

        return "\n".join(context_parts)

    def get_memory_statistics(self) -> Dict[str, Any]:
        """Get comprehensive memory system statistics"""
        stats = {}

        # Count by memory type
        for memory_type in MemoryType:
            count = self.conn.execute("""
                SELECT COUNT(*) FROM memory_entries WHERE memory_type = ?
            """, (memory_type.value,)).fetchone()[0]
            stats[f"{memory_type.value}_count"] = count

        # Working memory stats
        stats["working_memory_size"] = len(self.working_memory)
        stats["working_memory_max"] = self.max_working_memory

        # Access patterns
        most_accessed = self.conn.execute("""
            SELECT content, access_count FROM memory_entries
            ORDER BY access_count DESC LIMIT 3
        """).fetchall()
        stats["most_accessed"] = most_accessed

        # Recent activity
        recent_count = self.conn.execute("""
            SELECT COUNT(*) FROM memory_entries
            WHERE timestamp > ?
        """, ((datetime.now() - timedelta(hours=24)).isoformat(),)).fetchone()[0]
        stats["recent_memories"] = recent_count

        return stats

class ContextOptimizer:
    """Optimize context for agent prompts based on memory and current task"""

    def __init__(self, memory_manager: MemoryManager, max_context_length: int = 4000):
        self.memory_manager = memory_manager
        self.max_context_length = max_context_length

    def optimize_context(self, query: str, conversation_history: List[Dict] = None) -> str:
        """Create optimized context for current query"""
        context_parts = []
        current_length = 0

        # Add conversation history (most recent first)
        if conversation_history:
            context_parts.append("=== RECENT CONVERSATION ===")
            for entry in reversed(conversation_history[-3:]):  # Last 3 exchanges
                entry_text = f"User: {entry.get('user', '')}\nAssistant: {entry.get('assistant', '')}\n"
                if current_length + len(entry_text) < self.max_context_length // 2:
                    context_parts.append(entry_text)
                    current_length += len(entry_text)

        # Add relevant memories
        memory_context = self.memory_manager.get_memory_context(query)
        if current_length + len(memory_context) < self.max_context_length:
            context_parts.append(memory_context)
            current_length += len(memory_context)
        else:
            # Truncate memory context if needed
            available_space = self.max_context_length - current_length - 100  # Buffer
            if available_space > 200:
                truncated_context = memory_context[:available_space] + "... [truncated]"
                context_parts.append(truncated_context)

        return "\n\n".join(context_parts)


**# SECTION 3: Integration - Memory-Enhanced Planning Agent**

In [ ]:
print("\n" + "="*60)
print("SECTION 3: MEMORY-ENHANCED PLANNING AGENT")
print("="*60)

# Import tools from previous notebook (simplified versions)
@tool
def web_search_tool(query: str) -> str:
    """Search the web for information"""
    # Simplified web search simulation
    search_results = {
        "quantum computing": "Recent advances in quantum error correction and IBM's 1000-qubit roadmap",
        "AI development": "Latest developments in transformer architectures and multimodal AI systems",
        "stock market": "Current market trends show volatility in tech stocks with inflation concerns",
        "climate change": "New IPCC report highlights urgent need for emission reductions"
    }

    for topic, result in search_results.items():
        if topic.lower() in query.lower():
            return f"Search results for '{query}': {result}"

    return f"Search results for '{query}': General information found, but no specific match in simulation."

@tool
def calculator_tool(expression: str) -> str:
    """Perform calculations"""
    try:
        # Simple whitelist for safety
        allowed_chars = set('0123456789+-*/()., ')
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters in expression"

        result = eval(expression, {"__builtins__": {}}, {})
        return f"Calculation result: {result}"
    except Exception as e:
        return f"Calculation error: {e}"

@tool
def data_analysis_tool(data_description: str) -> str:
    """Analyze data trends and patterns"""
    analysis_responses = {
        "trend": "Analysis shows upward trend with 15% growth over the period",
        "correlation": "Strong positive correlation (r=0.85) between variables",
        "forecast": "Predictive model indicates continued growth with 70% confidence",
        "comparison": "Comparative analysis reveals significant differences between groups"
    }

    for key, response in analysis_responses.items():
        if key in data_description.lower():
            return f"Data analysis: {response}"

    return f"Data analysis completed for: {data_description}"

class MemoryEnhancedPlanningAgent:
    """Advanced agent combining planning, execution, and memory management"""

    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.llm = ChatOpenAI(model=model_name, temperature=0.1)
        self.memory_manager = MemoryManager()
        self.context_optimizer = ContextOptimizer(self.memory_manager)
        self.query_decomposer = QueryDecomposer(self.llm)

        # Available tools
        self.tools = [web_search_tool, calculator_tool, data_analysis_tool]
        self.plan_executor = PlanExecutor(self.llm, self.tools)

        # Conversation history
        self.conversation_history = []

    def process_query(self, query: str, use_memory: bool = True, adaptive_planning: bool = True) -> Dict[str, Any]:
        """Process query with full planning, execution, and memory integration"""
        print(f"\n🎯 Processing query: {query}")

        # Store query in working memory
        if use_memory:
            self.memory_manager.store_memory(
                MemoryType.WORKING,
                f"User query: {query}",
                {"timestamp": datetime.now().isoformat(), "type": "user_query"},
                ["query", "user_input"]
            )

        # Get optimized context
        context = self.context_optimizer.optimize_context(query, self.conversation_history) if use_memory else ""

        # Create execution plan
        print("📋 Creating execution plan...")
        plan = self.query_decomposer.decompose_query(query)

        # Store plan in memory
        if use_memory:
            self.memory_manager.store_memory(
                MemoryType.WORKING,
                f"Execution plan: {len(plan.steps)} steps",
                {"plan_type": plan.query_type.value, "steps": len(plan.steps)},
                ["plan", "execution"]
            )

        # Execute plan
        print("⚡ Executing plan...")
        execution_result = self.plan_executor.execute_plan(plan, adaptive=adaptive_planning)

        # Store execution results in memory
        if use_memory and execution_result["success"]:
            self.memory_manager.store_memory(
                MemoryType.EPISODIC,
                f"Successfully completed query: {query}",
                {
                    "query": query,
                    "steps_completed": execution_result["steps_completed"],
                    "total_time": execution_result.get("total_time", 0),
                    "final_answer": execution_result.get("final_synthesis", "")
                },
                ["success", "completion", plan.query_type.value]
            )

            # Store successful strategies as procedural memory
            if execution_result["steps_completed"] >= 2:
                successful_strategy = f"For {plan.query_type.value} queries, use {execution_result['steps_completed']}-step approach"
                self.memory_manager.store_memory(
                    MemoryType.PROCEDURAL,
                    successful_strategy,
                    {"query_type": plan.query_type.value, "success_rate": 1.0},
                    ["strategy", "procedure", "successful"]
                )

        # Update conversation history
        self.conversation_history.append({
            "user": query,
            "assistant": execution_result.get("final_synthesis", "No response generated"),
            "timestamp": datetime.now().isoformat()
        })

        # Periodic memory consolidation
        if len(self.conversation_history) % 5 == 0:  # Every 5 interactions
            self.memory_manager.consolidate_memory(self.llm)

        return {
            "query": query,
            "plan": plan,
            "execution_result": execution_result,
            "memory_context_used": bool(context),
            "conversation_turn": len(self.conversation_history)
        }

    def get_system_status(self) -> Dict[str, Any]:
        """Get comprehensive system status"""
        return {
            "memory_stats": self.memory_manager.get_memory_statistics(),
            "conversation_turns": len(self.conversation_history),
            "execution_history": len(self.plan_executor.execution_history),
            "active_plans": len(self.plan_executor.active_plans)
        }

    def export_memory_insights(self) -> Dict[str, Any]:
        """Export insights from accumulated memory"""
        insights = {
            "successful_strategies": [],
            "common_patterns": [],
            "learned_facts": [],
            "performance_metrics": {}
        }

        # Get procedural memories (strategies)
        procedural_memories = self.memory_manager.retrieve_memory(
            "strategy procedure",
            [MemoryType.PROCEDURAL],
            limit=10
        )
        insights["successful_strategies"] = [mem.content for mem in procedural_memories]

        # Get semantic memories (facts and patterns)
        semantic_memories = self.memory_manager.retrieve_memory(
            "pattern fact",
            [MemoryType.SEMANTIC],
            limit=10
        )
        for mem in semantic_memories:
            if "Pattern:" in str(mem.content):
                insights["common_patterns"].append(mem.content)
            elif "Fact:" in str(mem.content):
                insights["learned_facts"].append(mem.content)

        # Calculate performance metrics
        execution_history = self.plan_executor.execution_history
        if execution_history:
            success_rate = sum(1 for exec in execution_history if exec["success"]) / len(execution_history)
            avg_steps = sum(exec["steps_completed"] for exec in execution_history) / len(execution_history)
            avg_time = sum(exec.get("total_time", 0) for exec in execution_history) / len(execution_history)

            insights["performance_metrics"] = {
                "success_rate": success_rate,
                "average_steps_per_query": avg_steps,
                "average_execution_time": avg_time,
                "total_queries_processed": len(execution_history)
            }

        return insights

**# SECTION 4: Demonstration and Testing**

In [ ]:
print("\n" + "="*60)
print("SECTION 4: DEMONSTRATION AND TESTING")
print("="*60)

# Initialize the memory-enhanced planning agent
print("🚀 Initializing Memory-Enhanced Planning Agent...")
agent = MemoryEnhancedPlanningAgent()

# Test queries to demonstrate multi-step reasoning and memory
test_queries = [
    {
        "query": "What are the latest developments in quantum computing and how might they impact the semiconductor industry?",
        "description": "Complex research query requiring multi-step investigation and synthesis"
    },
    {
        "query": "If a quantum computer can perform certain calculations 1000x faster than classical computers, and a classical computer takes 10 hours for a specific task, how long would the quantum computer take? What are the implications for cryptography?",
        "description": "Mixed analytical query combining calculation and reasoning"
    },
    {
        "query": "Based on our previous discussion about quantum computing, what should investors consider when evaluating quantum technology companies?",
        "description": "Context-dependent query that should leverage memory"
    }
]

# Execute test queries
print("\n🧪 Running test queries...")
for i, test in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"TEST QUERY {i}: {test['description']}")
    print(f"{'='*60}")

    result = agent.process_query(test["query"])

    if result["execution_result"]["success"]:
        print(f"\n✅ Query completed successfully!")
        print(f"📋 Plan type: {result['plan'].query_type.value}")
        print(f"🔧 Steps executed: {result['execution_result']['steps_completed']}")
        print(f"⏱️ Execution time: {result['execution_result'].get('total_time', 0):.1f}s")
        print(f"🧠 Memory context used: {result['memory_context_used']}")
        print(f"\n💡 Final Answer:\n{result['execution_result']['final_synthesis']}")
    else:
        print(f"\n❌ Query failed: {result['execution_result'].get('error', 'Unknown error')}")

    # Show system status
    status = agent.get_system_status()
    print(f"\n📊 System Status:")
    print(f"   Memory entries: {sum(status['memory_stats'][k] for k in status['memory_stats'] if k.endswith('_count'))}")
    print(f"   Conversation turns: {status['conversation_turns']}")
    print(f"   Working memory: {status['memory_stats']['working_memory_size']}/{status['memory_stats']['working_memory_max']}")


**# SECTION 5: Memory Analysis and Visualization**

In [ ]:
print("\n" + "="*60)
print("SECTION 5: MEMORY ANALYSIS AND VISUALIZATION")
print("="*60)

def visualize_memory_distribution(memory_manager: MemoryManager):
    """Visualize memory distribution across types"""
    stats = memory_manager.get_memory_statistics()

    memory_types = []
    counts = []

    for memory_type in MemoryType:
        key = f"{memory_type.value}_count"
        if key in stats:
            memory_types.append(memory_type.value.title())
            counts.append(stats[key])

    if not counts:
        print("No memory data to visualize")
        return

    # Create visualization
    fig = go.Figure(data=[
        go.Bar(
            x=memory_types,
            y=counts,
            marker_color=['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728'][:len(counts)]
        )
    ])

    fig.update_layout(
        title="Memory Distribution by Type",
        xaxis_title="Memory Type",
        yaxis_title="Number of Entries",
        showlegend=False
    )

    fig.show()

    return fig

def analyze_planning_performance(plan_executor: PlanExecutor):
    """Analyze planning and execution performance"""
    history = plan_executor.execution_history

    if not history:
        print("No execution history to analyze")
        return

    # Extract performance data
    success_rates = []
    execution_times = []
    steps_completed = []

    for exec in history:
        success_rates.append(1 if exec["success"] else 0)
        execution_times.append(exec.get("total_time", 0))
        steps_completed.append(exec["steps_completed"])

    # Create performance dashboard
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=('Success Rate Over Time', 'Execution Time Distribution',
                       'Steps Completed Distribution', 'Performance Correlation'),
        specs=[[{"secondary_y": False}, {"secondary_y": False}],
               [{"secondary_y": False}, {"secondary_y": False}]]
    )

    # Success rate over time
    fig.add_trace(
        go.Scatter(y=success_rates, mode='lines+markers', name='Success Rate'),
        row=1, col=1
    )

    # Execution time distribution
    fig.add_trace(
        go.Histogram(x=execution_times, name='Execution Time'),
        row=1, col=2
    )

    # Steps completed distribution
    fig.add_trace(
        go.Histogram(x=steps_completed, name='Steps Completed'),
        row=2, col=1
    )

    # Performance correlation
    fig.add_trace(
        go.Scatter(x=steps_completed, y=execution_times, mode='markers',
                  name='Steps vs Time'),
        row=2, col=2
    )

    fig.update_layout(height=600, showlegend=False, title_text="Planning Performance Analysis")
    fig.show()

    # Print summary statistics
    print(f"\n📊 Performance Summary:")
    print(f"   Overall success rate: {sum(success_rates)/len(success_rates):.2%}")
    print(f"   Average execution time: {sum(execution_times)/len(execution_times):.1f}s")
    print(f"   Average steps per query: {sum(steps_completed)/len(steps_completed):.1f}")
    print(f"   Total queries processed: {len(history)}")

    return fig

# Generate visualizations
print("📊 Generating memory distribution visualization...")
memory_viz = visualize_memory_distribution(agent.memory_manager)

print("\n📈 Analyzing planning performance...")
performance_viz = analyze_planning_performance(agent.plan_executor)

# Export memory insights
print("\n🧠 Exporting memory insights...")
insights = agent.export_memory_insights()

print("\n💡 Memory Insights Summary:")
print(f"   Successful strategies learned: {len(insights['successful_strategies'])}")
print(f"   Common patterns identified: {len(insights['common_patterns'])}")
print(f"   Facts accumulated: {len(insights['learned_facts'])}")

if insights['performance_metrics']:
    metrics = insights['performance_metrics']
    print(f"   Current success rate: {metrics['success_rate']:.2%}")
    print(f"   Average query complexity: {metrics['average_steps_per_query']:.1f} steps")


**# SECTION 6: Advanced Memory Patterns**

In [ ]:
print("\n" + "="*60)
print("SECTION 6: ADVANCED MEMORY PATTERNS")
print("="*60)

class MemoryPattern:
    """Advanced memory pattern detection and utilization"""

    def __init__(self, memory_manager: MemoryManager):
        self.memory_manager = memory_manager

    def detect_query_patterns(self) -> Dict[str, List[str]]:
        """Detect patterns in user queries"""
        # Get episodic memories (completed interactions)
        episodic_memories = self.memory_manager.retrieve_memory(
            "query", [MemoryType.EPISODIC], limit=50
        )

        patterns = {
            "recurring_topics": [],
            "query_complexity_trends": [],
            "successful_approaches": []
        }

        # Simple pattern detection (in production, use more sophisticated NLP)
        topic_counts = {}
        for memory in episodic_memories:
            content = str(memory.content).lower()
            # Extract potential topics
            if "quantum" in content:
                topic_counts["quantum_computing"] = topic_counts.get("quantum_computing", 0) + 1
            if "market" in content or "stock" in content:
                topic_counts["financial_analysis"] = topic_counts.get("financial_analysis", 0) + 1
            if "ai" in content or "machine learning" in content:
                topic_counts["artificial_intelligence"] = topic_counts.get("artificial_intelligence", 0) + 1

        # Sort by frequency
        sorted_topics = sorted(topic_counts.items(), key=lambda x: x[1], reverse=True)
        patterns["recurring_topics"] = [f"{topic}: {count} occurrences" for topic, count in sorted_topics]

        return patterns

    def suggest_proactive_insights(self, llm) -> List[str]:
        """Generate proactive insights based on memory patterns"""
        patterns = self.detect_query_patterns()

        if not patterns["recurring_topics"]:
            return ["No sufficient interaction history for proactive insights"]

        insight_prompt = f"""
        Based on the user's interaction patterns, suggest proactive insights or recommendations.

        Recurring topics: {patterns["recurring_topics"]}

        Provide 3-5 actionable insights or suggestions that might be valuable to the user.
        Focus on:
        1. Emerging trends in their areas of interest
        2. Connections between different topics they've explored
        3. Potential next steps for their research or analysis

        Format as a simple list.
        """

        try:
            response = llm.invoke(insight_prompt)
            insights = [line.strip() for line in response.content.split('\n') if line.strip() and not line.strip().startswith('#')]
            return insights[:5]  # Limit to 5 insights
        except Exception as e:
            return [f"Could not generate proactive insights: {e}"]

# Test advanced memory patterns
print("🧠 Testing advanced memory patterns...")
memory_pattern_analyzer = MemoryPattern(agent.memory_manager)

patterns = memory_pattern_analyzer.detect_query_patterns()
print(f"\n🔍 Detected Patterns:")
for pattern_type, pattern_list in patterns.items():
    print(f"   {pattern_type.replace('_', ' ').title()}:")
    for pattern in pattern_list[:3]:  # Show top 3
        print(f"     - {pattern}")

print(f"\n💡 Generating proactive insights...")
proactive_insights = memory_pattern_analyzer.suggest_proactive_insights(agent.llm)
print("   Proactive Insights:")
for insight in proactive_insights:
    print(f"     - {insight}")


**# SECTION 7: Key Takeaways and Next Steps**

In [ ]:
print("\n" + "="*60)
print("KEY TAKEAWAYS AND NEXT STEPS")
print("="*60)

print("""
🎯 Key Takeaways from Notebook 13.2:

1. QUERY DECOMPOSITION & PLANNING:
   - Sophisticated query classification enables appropriate planning strategies
   - Dependency-aware execution order optimizes resource utilization
   - Adaptive planning allows recovery from failed steps

2. MEMORY SYSTEMS:
   - Multi-type memory (working, episodic, semantic, procedural) captures different knowledge
   - Automatic consolidation extracts patterns and insights from interactions
   - Context optimization balances relevance with computational constraints

3. INTEGRATION BENEFITS:
   - Memory-enhanced planning learns from past successes and failures
   - Context-aware execution improves response quality and consistency
   - Pattern detection enables proactive assistance

4. PERFORMANCE CONSIDERATIONS:
   - Memory management requires balance between retention and performance
   - Consolidation processes add computational overhead but improve long-term capability
   - Context length limits require intelligent selection and truncation

🚀 Next Steps:
- Explore multi-agent collaboration patterns in Notebook 13.3
- Implement production-ready memory backends (Redis, vector databases)
- Develop more sophisticated pattern recognition and learning algorithms
- Build evaluation frameworks for memory-enhanced agents

📊 Current System Performance:
""")

# Final system status
final_status = agent.get_system_status()
final_insights = agent.export_memory_insights()

print(f"- Total Queries Processed: {final_insights['performance_metrics'].get('total_queries_processed', 0)}")
print(f"- Memory Entries: {sum(final_status['memory_stats'][k] for k in final_status['memory_stats'] if k.endswith('_count'))}")
print(f"- Success Rate: {final_insights['performance_metrics'].get('success_rate', 0):.2%}")
print(f"- Strategies Learned: {len(final_insights['successful_strategies'])}")
print(f"- Patterns Identified: {len(final_insights['common_patterns'])}")

print(f"\n✨ Notebook Complete! Ready for multi-agent collaboration and production deployment.")